<a href="https://colab.research.google.com/github/NbtKmy/gc_workshops/blob/main/ChatGPT_Anwendungsbeispiel_ZB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatGPT mit Embedding

KI lügt - das Phänomen "Halluzination" ist mittlerweile bekannt. Auch wenn LLMs immer besser werden, bleibt sie weiterhin noch ein grosses Probelem.
Wenn Bibliotheken einen Dienstleistung mit ein LLM wie ChatGPT konzipieren, ist eine Massnahme gegen die Halluzination notwendig.

Hier in diesem Beispiel versuchen wir die Halluzination zu reduzieren, indem wir Embedding verwenden.
(Die andere denkbare Massnahme ist Fine Tuning. Aber hier wird sie nicht berücksichtigt.)

## Embedding?

>Bei einer Worteinbettung (englisch word embedding) handelt es sich um eine Einbettung, bei der Worte oder andere Symbole jeweils einem Vektor v mit $v \in \mathbb{R}^n$ zugeordnet werden.

(Wikipeida [Worteinbettung](https://de.wikipedia.org/wiki/Worteinbettung))

... Worteinbettung (Vektorisierung der Wörter) ermöglicht die Berechnung der Bedeutungen der Wörter in einem Kopus...

__Illustration für Worteinbettung__

<img src="https://upload.wikimedia.org/wikipedia/commons/3/3f/Word_vector_illustration.jpg" width="400" />

(Word vector illustration.jpg by Singerep, aus [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Word_vector_illustration.jpg) CC-SA)



Die Embedding kann nicht nur Wörtern sondern auch Sätze/Text-Einheiten unterzogen werden.



Was wir hier in diesem Beispiel machen, sieht so aus:



<img src="https://nbtkmy.github.io/presentations/coffee_lectures/chatgpt_embedding.png" alt="embedding" width="400"/>


## Beispiel


Als Beispiel nehmen wir das PDF-Dokument vom 1. Bericht der ZB.

__Titel__: 1. Bericht der Zentralbibliothek Zürich (Öffentliche Stiftung) über die Jahre 1914-1917

__Urheber__: Zentralbibliothek Zürich


__Link__: https://doi.org/10.20384/zop-1421


![zb](https://zop.zb.uzh.ch/retrieve/e7ca61c6-c25b-407e-b0f0-d401efa961f5/990019340740205508-0001.pdf.jpg)





In [ ]:
# Libraries installieren - Wir verwenden hier Langchain
!pip install -q openai chromadb langchain pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.4/422.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 120.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done

In [ ]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader


In [ ]:
# PDF-Dokument nach der Seite trennen (man kann auch andere Einheit auswählen)
loader = PyPDFLoader("./zb_1914_1917.pdf")
pages = loader.load_and_split()
# Seite 5 anzeigen
pages[10].page_content

'— —\nMit der Murforschenden Gesellschuft war, abgesehen von\nciner Abredeé betreffend die Pensionsverhältnisse ihres Ab-—\nwartes, die Frage der aus Zeitschrifteon und Einzelwerken\nbeſtehenden Deposita zuregeln, die eine Reihe von In—\ntuten der beiden Hochschulen aus der Bibliothek zustän-\ndigem Gebrauch bezogen hatten und weéeiterhin zubéhalten\nwünschten. Die Zentralbibliothek erklärte sich bereit, diese\nDeposita anzuerkennen inder Meinung, dab ihr das Recht\nzustehe, siejederzeit zuKontrollieren und allfällis von anderer\nSeite verlangte Bände vorũbergehend zurückzuziehen oder\nderen Durchsicht dureh diebetreffenden Interessenten amDepo-\nitionsort zuverlangen. Der Umfang dieser Deposita, die von\nder Zentralbibliothek einer genauen Bestandesaufnahme unter-\nzogen wurden, beträgt gegenwärtig 3850 Bände, wozu noch\n773 alsDeposita der ehemaligen Kantonsbibliothek kommen.\nPin veiteres Verhandlungspunkt betreffend dieUnterbringung\ndesGeésellschaftsarchiys und desStocks derPublik

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [ ]:
# Wenn kein Modell hier genannt, dann wird "text-embedding-ada-002" genommen.
# Chroma wird hier als Vector Store verwendet
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(pages, embedding=embeddings, persist_directory=".")
vectorstore.persist()

In [ ]:
pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)


In [ ]:
query = "Wer ist Escher?"
chat_history = []

result = pdf_qa({"question": query, "chat_history": chat_history})

result["answer"]

'Escher ist Dr. Hermann Escher, der Direktor der Bibliothek.'

In [ ]:
result["source_documents"]

[Document(page_content='Bibliothekkommission:\nHerren Reégierungsrat Dr. Heinr. Mousson (Präsident),\nStadtpräsident Hans Nägeli (Vizepräsident), Prof. Dr —\nCloetta, Prof. Dr. Aug. Pgger, Prof. Dr. Fritz Bleiner, Stadtrat\nDr.Emil Rlöti, Prof. Dr.Gerold Meyer vonKnonau, Prof. Dr.\nMartin Rikli, Prof. Dr. Férd. Rudio, Prof. Dr. Hans Schinz,\nOberst Pduard Usteri-Pestalozzi, Rektor Dr.Wilhelm vonVyb,\nDr.Hermann Escher (Aktuar), Dr.Hans Barth Erotokollfũhrer).\nAusschuſ:\nHerren Mousson (Präsident), Nägeli (Vizepräsident), Egger,\nv.WMyb, Escher, Barth.\nRechnungskommission:\nHerren Usteri (Präsident), Prof. Dr. G.Bachmann, Nat-\nRat. John 82.\nKommission fürdieStiftung Schnyder von Martensee:\nHerren Stadtpräsident Hans NJägeli (Präsident), Dr.Her-\nmann Escher (Aktuar), Prof. Dr. G.Meyer von Knonau, Prof.\nDr. Hans Schinz, Oberst E.Usteri-Pestalozzi.\nNaturwissenschaftliche Subhkommission:\nHerren Prof. Dr. Albert Heine (Präsident); Rektor Dr.\nE.Bobhard, Prof. Dr. O.Schröter.\nperson